In [31]:
import simpy
import random
import statistics

In [43]:
wait_times = []

class SecurityCheck(object):
    def __init__(self, env, num_passcheck, num_bag_lines):
        self.env = env
        self.passport_checker = simpy.Resource(env, num_passcheck)
        self.baggage_scan_line = simpy.Resource(env, num_bag_lines)

    def get_serviced(self, traveler):
        yield self.env.timeout(random.expovariate(0.75))

    def personal_check(self, traveler):
        yield self.env.timeout(random.uniform(0.5,1))

In [44]:
def go_through_security(env, traveler, airport):
    #traveler arrives at the security line
    arrival_time = env.now

    with airport.passport_checker.request() as request:
        yield request
        yield env.process(airport.get_serviced(traveler))

    with airport.baggage_scan_line.request() as request:
        yield request
        yield env.process(airport.personal_check(traveler))

    #traveler clears all of security
    wait_times.append(env.now - arrival_time)

In [45]:
def run_airport(env, num_passcheck, num_bag_lines):
    airport = SecurityCheck(env, num_passcheck, num_bag_lines)

    for traveler in range(1):
        env.process(go_through_security(env, traveler, airport))

    while True:
        yield env.timeout(0.020)
        traveler += 1
        env.process(go_through_security(env, traveler, airport))

In [46]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)

In [47]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [48]:
def get_user_input():
    num_passcheck = input("Input # of Passport Checkers working: ")
    num_bag_lines = input("Input # of Bag Lines operating: ")
    params = [num_passcheck, num_bag_lines]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print(
            "Could not parse input. The simulation will use default values:",
            "\n1 PassCheck, 1 BagLine.",
        )
        params = [1, 1]
    return params

In [49]:
def main():
  # Setup
  random.seed(101)
  num_passcheck, num_bag_lines = get_user_input()

  # Run the simulation
  env = simpy.Environment()
  env.process(run_airport(env, num_passcheck, num_bag_lines))
  env.run(until=120)

  # View the results
  mins, secs = get_average_wait_time(wait_times)
  print(
      "Running simulation...",
      f"\nThe average wait time is {mins} minutes and {secs} seconds.",
  )

In [56]:
if __name__ == '__main__':
    main()

Input # of Passport Checkers working: 100
Input # of Bag Lines operating: 100
Running simulation... 
The average wait time is 14 minutes and 13 seconds.
